In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.renderers.default = "iframe"

  #   url to pull information from
url = ' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html.parser')  #  use beautiful soup library
  #  create our empty dataframe
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])  #use pandas dataframe function to structure information

for row in soup.find("tbody").find_all('tr'):  #   use beautifulsoup to parse needed information
    col = row.find_all("td")
    date = col[0].text
    Revenue = col[1].text
    gme_revenue = pd.concat([gme_revenue, pd.DataFrame(
        {"Date": [date], "Revenue": [Revenue]})], ignore_index=True)

gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(r',|\$',"", regex=True)  # append gathered information from gamestop_revenue to format needed

gme_revenue.dropna(inplace=True)  #remove missing values from information

gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]  # removes any rows that have no data under revenue column

print(gme_revenue.tail())  #  print the tail end of our gathered information (tail = print last 5 rows)

stock = yf.Ticker("TSLA")  # listing a stock for the yfinance function

# Fetch historical data with max period
stock_data = stock.history(period="max")

stock_data.reset_index(inplace=True)
stock_data = stock_data.drop('Open', axis=1)
stock_data = stock_data.drop('High', axis=1)
stock_data = stock_data.drop('Low', axis=1)
stock_head = stock_data.head()

print(f" {stock_head}")
#print(stock_data.columns)



def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    #fig_html = fig.to_html()
    #display(HTML(fig_html))

make_graph(stock_head, gme_revenue,'Gamestop')

    Date Revenue
11  2009    8806
12  2008    7094
13  2007    5319
14  2006    3092
15  2005    1843
                        Date     Close     Volume  Dividends  Stock Splits
0 2010-06-29 00:00:00-04:00  1.592667  281494500        0.0           0.0
1 2010-06-30 00:00:00-04:00  1.588667  257806500        0.0           0.0
2 2010-07-01 00:00:00-04:00  1.464000  123282000        0.0           0.0
3 2010-07-02 00:00:00-04:00  1.280000   77097000        0.0           0.0
4 2010-07-06 00:00:00-04:00  1.074000  103003500        0.0           0.0


/tmp/ipykernel_2422/3364827957.py:53: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

/tmp/ipykernel_2422/3364827957.py:54: UserWarning:

The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.

